<a href="https://colab.research.google.com/github/yimstar9/Engineer-Big-Data-Analysis_R/blob/main/%5BR%5D4th_type2_score_0_32166_(3models_RF%2CXGB%2Cnnet).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages
library(dplyr)
library(xgboost)
#library(ModelMetrics)
library(caret)
library(nnet)
library(randomForest)

df_train<-read.csv("../input/big-data-analytics-certification-kr-2022/train.csv")
df_test<-read.csv("../input/big-data-analytics-certification-kr-2022/test.csv")

####전처리#### ###
train<-df_train[,-1]
test<-df_test[,-1]
train <- train %>% mutate(Age=(Age%/%10)*10)%>%mutate_all(as.factor)
test <- test %>%mutate(Age=(Age%/%10)*10)%>% mutate_all(as.factor)

#훈련, 테스트셋 나눔
idx<-sample(1:nrow(train),0.7*nrow(train))
x_train<-train[idx,]
x_test<-train[-idx,]


###################age,profession,var_1,Family_Size 원핫인코딩########################
###종속변수만 factor형 나머지 전부 num형 -> f1_macro : 0.31602
# train <- train %>%mutate(Age=(Age%/%10)*10)%>%mutate_at(vars(Age,Profession,Var_1,Family_Size),as.factor)
# test <- test %>%mutate(Age=(Age%/%10)*10)%>%mutate_at(vars(Age,Profession,Var_1,Family_Size),as.factor)

# dummy <- dummyVars("~.", data = train)
# train <- data.frame(predict(dummy, newdata = train))
# dummy2 <- dummyVars("~.", data = test)
# test <- data.frame(predict(dummy2, newdata = test))





str(x_train)
str(x_test)
# 훈련 모델 생성, 예측,평가

m_rf<-randomForest(Segmentation~.,x_train,ntree=300)
p_rf<-predict(m_rf,subset(x_test, select=-Segmentation))
m_net <- nnet(Segmentation ~ ., x_train, size = 3)
p_net <- predict(m_net, subset(x_test, select=-Segmentation), type = "class")

print("randomforest")
caret::confusionMatrix(x_test$Segmentation,p_rf)$overall[1]
caret::confusionMatrix(x_test$Segmentation,p_rf)$byClass[7]

print("nnet")
caret::confusionMatrix(x_test$Segmentation,as.factor(p_net))$overall[1]
caret::confusionMatrix(x_test$Segmentation,as.factor(p_net))$byClass[7]


###########################최종모델 선택&제출############################
##########랜덤포레스트############
## 모든변수 factor형 -> f1_macro : 0.3209
# m2_rf<-randomForest(Segmentation~.,train,ntree=300)
# p2_rf<-predict(m2_rf,test)

# result<-cbind(as.character(df_test[,1]),as.character(p2_rf))
# colnames(result)<-c("ID","Segmentation")
# write.csv(result,"submission.csv",row.names=F)
# zz<-read.csv("submission.csv")
# head(zz)
# str(zz)


#################인공신경망###########
##f1_macro : 0.32166
# model_net = nnet(Segmentation ~ ., train, size = 3)
# p <- predict(model_net, test, type = "class")
# result<-cbind(as.character(df_test[,1]),as.character(p))
# colnames(result)<-c("ID","Segmentation")
# write.csv(result,"submission.csv",row.names=F)
# zz<-read.csv("submission.csv")
# head(zz)
# str(zz)


################xgboost#######################
###f1_macro : 0.30264

##xgboost용 데이터
# xg_train<-train
# xg_test<-test

# ############xgboost모델링
# train_mat <- as.matrix(subset(xg_train, select=-Segmentation))
# test_mat<- as.matrix(xg_test)
# train_lab <- train$Segmentation
# dtrain <- xgb.DMatrix(data = train_mat, label = train_lab)

# xgb_model <- xgboost(data = dtrain, max_depth = 15, eta = 0.3,nthread = 3, nrounds = 25,objective = "multi:softmax",num_class = 5,verbose = 0)
# xgb_pred <- predict(xgb_model, test_mat)
# head(xgb_pred)

# result<-cbind(as.character(df_test[,1]),as.character(xgb_pred))
# colnames(result)<-c("ID","Segmentation")
# write.csv(result,"submission.csv",row.names=F)
# zz<-read.csv("submission.csv")
# head(zz)
# str(zz)
# print("finish")